# 모두의 딥러닝 개정 3판 
## 19장 세상에 없는 얼굴 GAN, 오토인코더
- 생성적 적대 신경망(Generative Adversarial Networks, GAN(간))

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import models, layers
import numpy as np
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
# 생성자 모델 만들기

generator = models.Sequential()
# 7*7 이미지 채널 128개짜리 만들기 위해 식으로 표현
# layers.LeakyReLU(0.2) '' 이 아니라 직접 쓴 이유, 이후에 하이퍼 파라미터 만들기 위해
generator.add(layers.Dense(7*7*128, input_dim=100,
              activation=layers.LeakyReLU(0.2)))
# Dense 출력이 BatchNormalization 입력으로 들어와 평균=0, 표준편차=1로 스케일링해줌
generator.add(layers.BatchNormalization())

generator.add(layers.Reshape((7, 7, 128)))
# UpSampling2D: MaxPooling 정반대 7*7 -> 14*14로 늘리는 중
generator.add(layers.UpSampling2D())

# padding='same': 원본 사이즈 유지하겠다
# kernel_size=5이니 상하좌우 2줄씩 0으로
generator.add(layers.Conv2D(64, kernel_size=5, padding='same'))
generator.add(layers.BatchNormalization())
generator.add(layers.Activation(layers.LeakyReLU(0.2)))
generator.add(layers.UpSampling2D())
generator.add(layers.Conv2D(1, kernel_size=5, padding='same', activation='tanh'))
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6272)              633472    
                                                                 
 batch_normalization (BatchN  (None, 6272)             25088     
 ormalization)                                                   
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 14, 14, 128)      0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 64)        204864    
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 64)       2

In [3]:
# 판별자 모델 만들기

discriminator = models.Sequential()
# strides=2; 2픽셀씩 확인(듬성듬성 특징을 뽑아내겠다, 계산량 줄어듬, 세밀함 줄어듬)
discriminator.add(layers.Conv2D(64, kernel_size=5, input_shape=(28,28,1), strides=2, padding='same'))
discriminator.add(layers.Activation(layers.LeakyReLU(0.2)))
discriminator.add(layers.Dropout(0.3))
discriminator.add(layers.Conv2D(128, kernel_size=5, strides=2, padding='same'))
discriminator.add(layers.Activation(layers.LeakyReLU(0.2)))
discriminator.add(layers.Dropout(0.3))
discriminator.add(layers.Flatten())
discriminator.add(layers.Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False
# 생성자가 가짜 noise를 던지면 순전파를 통해 판별자에게 옴
# 판별자는 오차를 판별하여 다시 생성자에게 보내면 생성자는 돌려받은 것으로 파라미터 갱신
# 그런데 이때 생성자가 보낸 오류가 많은 정보로 학습을 하면 안되니 기본적으로 학습 미실시로 세팅
# 따로 학습시켜야함

discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        1664      
                                                                 
 activation_1 (Activation)   (None, 14, 14, 64)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 activation_2 (Activation)   (None, 7, 7, 128)         0         
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)             

In [5]:
ginput = layers.Input(shape=(100))
dis_output = discriminator(generator(ginput))
gan = models.Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 1)         865281    
                                                                 
 sequential_1 (Sequential)   (None, 1)                 212865    
                                                                 
Total params: 1,078,146
Trainable params: 852,609
Non-trainable params: 225,537
_________________________________________________________________
